# Code for data work for Medicaid/SNAP paper

In [45]:
# Import packages
import numpy as np
import pandas as pd
import os
import json
import pickle
import openpyxl
import geopandas as gpd
import matplotlib.pyplot as plt

from bokeh.io import output_file, output_notebook
from bokeh.palettes import Viridis256
from bokeh.plotting import figure, show
from bokeh.models import (ColumnDataSource, Title, Label, LabelSet, Legend,
                          LegendItem, CategoricalColorMapper, ColorBar,
                          HoverTool, NumeralTickFormatter, GeoJSONDataSource,
                          FactorRange)
from bokeh.models.tickers import SingleIntervalTicker
from bokeh.transform import factor_cmap

In [2]:
cur_dir = "/Users/richardevans/Docs/Economics/OSE/MedicaidSNAP"
data_dir = os.path.join(cur_dir, "data")

In [3]:
# Read in data from PovertyEstimates.xlsx file into pandas dataframe
df = pd.read_excel(
    os.path.join(data_dir, "PovertyEstimates.xlsx"),
    sheet_name="PovertyEstimates",
    header=4,
    usecols=[
        "FIPS_Code",
        "Stabr",
        "Area_name",
        "Rural-urban_Continuum_Code_2003",
        "Urban_Influence_Code_2003",
        "Rural-urban_Continuum_Code_2013",
        "Urban_Influence_Code_2013",
        "PCTPOVALL_2021",
        "PCTPOV017_2021",
        "PCTPOV517_2021",
        "MEDHHINC_2021",
    ]
)

In [4]:
# Change FIPS_Code variable to 5-digit string
df["FIPS_Code"] = df["FIPS_Code"].astype(str).str.zfill(5)

In [5]:
# Create a Python list of US state full names
US_states_list = [
    "Alabama",
    "Alaska",
    "Arizona",
    "Arkansas",
    "California",
    "Colorado",
    "Connecticut",
    "Delaware",
    "Florida",
    "Georgia",
    "Hawaii",
    "Idaho",
    "Illinois",
    "Indiana",
    "Iowa",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Maine",
    "Maryland",
    "Massachusetts",
    "Michigan",
    "Minnesota",
    "Mississippi",
    "Missouri",
    "Montana",
    "Nebraska",
    "Nevada",
    "New Hampshire",
    "New Jersey",
    "New Mexico",
    "New York",
    "North Carolina",
    "North Dakota",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Pennsylvania",
    "Rhode Island",
    "South Carolina",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Vermont",
    "Virginia",
    "Washington",
    "West Virginia",
    "Wisconsin",
    "Wyoming",
]

# Delete rows for Area_name equals United States or any of the US states alone
delete_list = ["United States"] + US_states_list
df = df[~df["Area_name"].isin(delete_list)]

# Delete the state entry for District of Columbia (FIPS_Code="11000"), but
# leave the county entry (FIPS_Code="11001")
df = df[df["FIPS_Code"] != "11000"]

In [6]:
# Merge in unemployment data
df_unemp = pd.read_excel(
    os.path.join(data_dir, "Unemployment.xlsx"),
    sheet_name="UnemploymentMedianIncome",
    header=4,
    usecols=[
        "FIPS_Code",
        "State",
        "Area_Name",
        "Rural_Urban_Continuum_Code_2013",
        "Urban_Influence_Code_2013",
        "Metro_2013",
        "Unemployment_rate_2021",
        "Unemployment_rate_2022"
    ]
)

# Rename column to differentiate from the df dataframe
df_unemp.rename(
    columns={
        "Urban_Influence_Code_2013": "Urban_Influence_Code_2013_unemp",
    },
    inplace=True
)

In [7]:
# Change FIPS_Code variable to 5-digit string
df_unemp["FIPS_Code"] = df_unemp["FIPS_Code"].astype(str).str.zfill(5)

# Delete rows for Area_name equals United States or any of the US states alone
df_unemp = df_unemp[~df_unemp["Area_Name"].isin(delete_list)]

# Drop if state is Puerto Rico (PR)
df_unemp = df_unemp[df_unemp["State"] != "PR"]

# Merge Metro_2013, Unemployment_rate_2021, and Unemployment_rate_2022 into df based on FIPS_Code
df = pd.merge(
    df,
    df_unemp[[
        "FIPS_Code",
        "Area_Name",
        "Rural_Urban_Continuum_Code_2013",
        "Urban_Influence_Code_2013_unemp",
        "Metro_2013",
        "Unemployment_rate_2021",
        "Unemployment_rate_2022"
    ]],
    on="FIPS_Code",
    how="outer",
    indicator=True
)
df.head()

,FIPS_Code,Stabr,Area_name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,PCTPOVALL_2021,PCTPOV017_2021,PCTPOV517_2021,MEDHHINC_2021,Area_Name,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013_unemp,Metro_2013,Unemployment_rate_2021,Unemployment_rate_2022,_merge
0,01001,AL,Autauga County,2.0,2.0,2.0,2.0,10.7,16.1,15.6,66444.0,"Autauga County, AL",2.0,2.0,1.0,2.8,2.3,both
1,01003,AL,Baldwin County,4.0,5.0,3.0,2.0,10.8,16.4,15.2,65658.0,"Baldwin County, AL",3.0,2.0,1.0,2.9,2.4,both
2,01005,AL,Barbour County,6.0,6.0,6.0,6.0,23.0,35.1,33.8,38649.0,"Barbour County, AL",6.0,6.0,0.0,5.5,4.1,both
3,01007,AL,Bibb County,1.0,1.0,1.0,1.0,20.6,29.0,29.0,48454.0,"Bibb County, AL",1.0,1.0,1.0,3.4,2.5,both
4,01009,AL,Blount County,1.0,1.0,1.0,1.0,12.0,16.7,15.9,56894.0,"Blount County, AL",1.0,1.0,1.0,2.4,2.2,both


In [8]:
df.describe()

,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,PCTPOVALL_2021,PCTPOV017_2021,PCTPOV517_2021,MEDHHINC_2021,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013_unemp,Metro_2013,Unemployment_rate_2021,Unemployment_rate_2022
count,3136.000000,3136.000000,3141.000000,3141.000000,3142.000000,3142.000000,3142.000000,3142.000000,3141.000000,3141.000000,3146.000000,3143.000000,3143.000000
mean,5.124362,5.446747,5.007323,5.265839,14.609866,20.050127,18.902355,58941.633355,5.009233,5.268704,0.370312,4.650843,3.594400
std,2.681944,3.468386,2.707905,3.498363,5.663115,8.413598,8.138269,15267.686038,2.708605,3.499373,0.482965,1.730044,1.231279
min,1.000000,1.000000,1.000000,1.000000,2.900000,2.800000,2.400000,25653.000000,1.000000,1.000000,0.000000,0.900000,0.600000
25%,3.000000,2.000000,2.000000,2.000000,10.600000,13.800000,12.800000,49016.250000,2.000000,2.000000,0.000000,3.500000,2.700000
50%,6.000000,5.000000,6.000000,5.000000,13.600000,19.100000,17.800000,56634.000000,6.000000,5.000000,0.000000,4.400000,3.400000
75%,7.000000,8.000000,7.000000,8.000000,17.575000,24.800000,23.400000,65687.250000,7.000000,8.000000,1.000000,5.500000,4.200000
max,9.000000,12.000000,9.000000,12.000000,43.900000,58.500000,61.100000,153716.000000,9.000000,12.000000,1.000000,19.500000,14.700000


In [9]:
df["_merge"].value_counts()

_merge
both          3142
right_only       5
left_only        1
Name: count, dtype: int64

In [10]:
df[["FIPS_Code", "Area_name", "Area_Name"]][df["_merge"] == "left_only"]

,FIPS_Code,Area_name,Area_Name
554,15005,Kalawao County,NaN


In [11]:
df[["FIPS_Code", "Area_name", "Area_Name"]][df["_merge"] == "right_only"]

,FIPS_Code,Area_name,Area_Name
91,02201,NaN,"Prince of Wales-Outer Ketchikan Census Area, AK"
94,02232,NaN,"Skagway-Hoonah-Angoon Census Area, AK"
96,02261,NaN,"Valdez-Cordova Census Area, AK"
98,02280,NaN,"Wrangell-Petersburg Census Area, AK"
324,11000,NaN,District of Columbia


In [12]:
# Replace Area_name with Area_Name for observations in which _merge is "right_only"
df.loc[df["_merge"] == "right_only", "Area_name"] = df.loc[df["_merge"] == "right_only", "Area_Name"]
df[["FIPS_Code", "Area_name", "Area_Name"]][df["_merge"] == "right_only"]

,FIPS_Code,Area_name,Area_Name
91,02201,"Prince of Wales-Outer Ketchikan Census Area, AK","Prince of Wales-Outer Ketchikan Census Area, AK"
94,02232,"Skagway-Hoonah-Angoon Census Area, AK","Skagway-Hoonah-Angoon Census Area, AK"
96,02261,"Valdez-Cordova Census Area, AK","Valdez-Cordova Census Area, AK"
98,02280,"Wrangell-Petersburg Census Area, AK","Wrangell-Petersburg Census Area, AK"
324,11000,District of Columbia,District of Columbia


In [13]:
df[["FIPS_Code", "Area_name", "Rural-urban_Continuum_Code_2013", "Rural_Urban_Continuum_Code_2013"]][df["_merge"] == "right_only"]

,FIPS_Code,Area_name,Rural-urban_Continuum_Code_2013,Rural_Urban_Continuum_Code_2013
91,02201,"Prince of Wales-Outer Ketchikan Census Area, AK",NaN,NaN
94,02232,"Skagway-Hoonah-Angoon Census Area, AK",NaN,NaN
96,02261,"Valdez-Cordova Census Area, AK",NaN,9.0
98,02280,"Wrangell-Petersburg Census Area, AK",NaN,NaN
324,11000,District of Columbia,NaN,NaN


In [14]:
# Replace ? with Rural_Urban_Continuum_Code_2013 for
# observations in which _merge is "right_only"
df.loc[df["_merge"] == "right_only", "Rural-urban_Continuum_Code_2013"] = df.loc[df["_merge"] == "right_only", "Rural_Urban_Continuum_Code_2013"]
df[["FIPS_Code", "Area_name", "Rural-urban_Continuum_Code_2013", "Rural_Urban_Continuum_Code_2013"]][df["_merge"] == "right_only"]

,FIPS_Code,Area_name,Rural-urban_Continuum_Code_2013,Rural_Urban_Continuum_Code_2013
91,02201,"Prince of Wales-Outer Ketchikan Census Area, AK",NaN,NaN
94,02232,"Skagway-Hoonah-Angoon Census Area, AK",NaN,NaN
96,02261,"Valdez-Cordova Census Area, AK",9.0,9.0
98,02280,"Wrangell-Petersburg Census Area, AK",NaN,NaN
324,11000,District of Columbia,NaN,NaN


In [15]:
df[["FIPS_Code", "Area_name", "Urban_Influence_Code_2013", "Urban_Influence_Code_2013_unemp"]][df["_merge"] == "right_only"]

,FIPS_Code,Area_name,Urban_Influence_Code_2013,Urban_Influence_Code_2013_unemp
91,02201,"Prince of Wales-Outer Ketchikan Census Area, AK",NaN,NaN
94,02232,"Skagway-Hoonah-Angoon Census Area, AK",NaN,NaN
96,02261,"Valdez-Cordova Census Area, AK",NaN,11.0
98,02280,"Wrangell-Petersburg Census Area, AK",NaN,NaN
324,11000,District of Columbia,NaN,NaN


In [16]:
# Replace Urban_Influence_Code_2013 with Urban_Influence_Code_2013_unemp for
# observations in which _merge is "right_only"
df.loc[df["_merge"] == "right_only", "Urban_Influence_Code_2013"] = df.loc[df["_merge"] == "right_only", "Urban_Influence_Code_2013_unemp"]
df[["FIPS_Code", "Area_name", "Urban_Influence_Code_2013", "Urban_Influence_Code_2013_unemp"]][df["_merge"] == "right_only"]

,FIPS_Code,Area_name,Urban_Influence_Code_2013,Urban_Influence_Code_2013_unemp
91,02201,"Prince of Wales-Outer Ketchikan Census Area, AK",NaN,NaN
94,02232,"Skagway-Hoonah-Angoon Census Area, AK",NaN,NaN
96,02261,"Valdez-Cordova Census Area, AK",11.0,11.0
98,02280,"Wrangell-Petersburg Census Area, AK",NaN,NaN
324,11000,District of Columbia,NaN,NaN


In [17]:
# Drop the _merge and Area_Name columns
df = df.drop(columns=["_merge", "Area_Name", "Rural_Urban_Continuum_Code_2013", "Urban_Influence_Code_2013_unemp"])
df.keys()

Index(['FIPS_Code', 'Stabr', 'Area_name', 'Rural-urban_Continuum_Code_2003',
       'Urban_Influence_Code_2003', 'Rural-urban_Continuum_Code_2013',
       'Urban_Influence_Code_2013', 'PCTPOVALL_2021', 'PCTPOV017_2021',
       'PCTPOV517_2021', 'MEDHHINC_2021', 'Metro_2013',
       'Unemployment_rate_2021', 'Unemployment_rate_2022'],
      dtype='object')

# Create Average Medicaid payout per participant data by state and add it to the county data

In [44]:
df_medicaid_state = pd.read_csv(
    os.path.join(data_dir, "kff_avg_medicaid_payout.csv"),
    header=2,
    usecols=[
        "Location",	"All Full or Partial Benefit Enrollees"
    ],
    skipfooter=24,
    engine="python"
)
# Rename columns
df_medicaid_state.rename(
    columns={
        "Location": "Area_name",
    },
    inplace=True
)
# Delete "United States" row
df_medicaid_state = df_medicaid_state[
    df_medicaid_state["Area_name"] != "United States"
]
# Create new column that is a numerical version of the All Full or Partial
# Benefit Enrollees column
df_medicaid_state["avg_nom_medicaid"] = df_medicaid_state[
    "All Full or Partial Benefit Enrollees"
].str.replace("$", "").str.replace(",", "").astype(float)
# Drop the All Full or Partial Benefit Enrollees column
df_medicaid_state = df_medicaid_state.drop(columns=[
    "All Full or Partial Benefit Enrollees"
])
# Get DataFrame with the FIPS_Code and two-digit state abbreviation
# Read in data from PovertyEstimates.xlsx file into pandas dataframe
df_code = pd.read_excel(
    os.path.join(data_dir, "PovertyEstimates.xlsx"),
    sheet_name="PovertyEstimates",
    header=4,
    usecols=[
        "FIPS_Code",
        "Stabr",
        "Area_name",
    ]
)
# Change FIPS_Code variable to 5-digit string
df_code["FIPS_Code"] = df_code["FIPS_Code"].astype(str).str.zfill(5)
# Merge in FIPS_Code and two-digit state abbreviation
df_medicaid_state = pd.merge(
    df_medicaid_state,
    df_code[["FIPS_Code", "Stabr", "Area_name"]],
    on="Area_name",
    how="left"
)
# Drop District of Columbia county (FIPS_Code="11001")
df_medicaid_state = df_medicaid_state[
    df_medicaid_state["FIPS_Code"] != "11001"
].reset_index(drop=True)
# Reorder columns
df_medicaid_state = df_medicaid_state[[
    "FIPS_Code",  "Stabr", "Area_name", "avg_nom_medicaid"
]]

,FIPS_Code,Stabr,Area_name,avg_nom_medicaid
0,01000,AL,Alabama,4777.0
1,02000,AK,Alaska,8640.0
2,04000,AZ,Arizona,5616.0
3,05000,AR,Arkansas,6480.0
4,06000,CA,California,5581.0
5,08000,CO,Colorado,6414.0
6,09000,CT,Connecticut,8063.0
7,10000,DE,Delaware,7701.0
8,11000,DC,District of Columbia,10166.0
9,12000,FL,Florida,4718.0


In [52]:
# Save the dataframe to a pickle file
df_medicaid_state.to_pickle(os.path.join(data_dir, "medicaid_snap_state_data.pkl"))

In [50]:
# Merge avg_nom_medicaid into df based on Stabr
df = pd.merge(
    df,
    df_medicaid_state[[
        "Stabr",
        "avg_nom_medicaid"
    ]],
    on="Stabr",
    how="left",
)

# Save the dataframe to a pickle file
df.to_pickle(os.path.join(data_dir, "county_data.pkl"))


In [ ]:
df = pickle.load(open(os.path.join(data_dir, "full_data.pkl"), "rb"))
# Save df as a csv file
df.to_csv(os.path.join(data_dir, "county_data.csv"), index=False)

# Add SNAP benefit for single parent of two to state and county datasets

# Create average Medicaid benefit payout map by state for 2019

In [53]:
# fig1_title = "Average Medicaid payouts by state, 2019"
fig1_title = ""
output_file(
    "./images/medicaid_state_2019.html", title=fig1_title, mode='inline'
)
output_notebook()

# Download U.S. states shape files from US Census Bureau
# https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html
us_state_shapefile_path = ("https://github.com/TheCGO/MN-AMT/raw/main/data/" +
                           "cb_2022_us_state_20m/cb_2022_us_state_20m.shp")
gdf = gpd.GeoDataFrame.from_file(us_state_shapefile_path)
gdf_json = gdf.to_json()
gjson = json.loads(gdf_json)
gjson["features"]

# # Remove Puerto Rico from data
# del(gjson["features"][7])

# # Alaska
# # Fix positive longitudes
# min_lat_ak = 180  # initial value that will be adjusted
# min_abs_lon_ak = 180  # initial value that will be adjusted
# coords_list = gjson["features"][24]["geometry"]["coordinates"]
# for ind_isl, island in enumerate(coords_list):
#     for ind_pnt, point in enumerate(island[0]):
#         min_lat_ak = np.minimum(min_lat_ak, point[1])
#         if point[0] > 0:
#             gjson["features"][24]["geometry"][
#                 "coordinates"
#             ][ind_isl][0][ind_pnt][0] = -180 - (180 - point[0])
#         else:
#             min_abs_lon_ak = np.minimum(min_abs_lon_ak, -point[0])

# # Shrink the size of Alaska relative to its southestern most minimum lattitude
# # and longitude
# shrink_pct_ak = 0.65
# coords_list_ak = gjson["features"][24]["geometry"]["coordinates"]
# for ind_isl, island in enumerate(coords_list_ak):
#     for ind_pnt, point in enumerate(island[0]):
#         gjson["features"][24]["geometry"][
#             "coordinates"
#         ][ind_isl][0][ind_pnt][0] = point[0] - shrink_pct_ak * (point[0] +
#                                                                 min_abs_lon_ak)
#         gjson["features"][24]["geometry"][
#             "coordinates"
#         ][ind_isl][0][ind_pnt][1] = point[1] - shrink_pct_ak * (point[1] -
#                                                                 min_lat_ak)

# # Move Alaska closer to the mainland such that the minimum minimum absolute
# # longitude and lattitude are (-127, 44)
# min_lat_ak_new = 44
# min_abs_lon_ak_new = 127
# for ind_isl, island in enumerate(coords_list):
#     for ind_pnt, point in enumerate(island[0]):
#         gjson["features"][24]["geometry"][
#             "coordinates"
#         ][ind_isl][0][ind_pnt][0] = point[0] + (min_abs_lon_ak -
#                                                 min_abs_lon_ak_new)
#         gjson["features"][24]["geometry"][
#             "coordinates"
#         ][ind_isl][0][ind_pnt][1] = point[1] - (min_lat_ak - min_lat_ak_new)

# # Hawaii
# list_ind_hi = 47
# # Get minimum lattitude and minimum absolute longitude for Hawaii
# min_lat_hi = 180  # initial value that will be adjusted
# min_abs_lon_hi = 180  # initial value that will be adjusted
# coords_list = gjson["features"][list_ind_hi]["geometry"]["coordinates"]
# for ind_isl, island in enumerate(coords_list):
#     for ind_pnt, point in enumerate(island[0]):
#         min_lat_hi = np.minimum(min_lat_hi, point[1])
#         min_abs_lon_hi = np.minimum(min_abs_lon_hi, -point[0])
# # print("Minimum lattitude for Hawaii is", min_lat_hi)
# # print("Minimum absolute longitude for Hawaii is", min_abs_lon_hi)

# # Increase the size of Hawaii
# incr_pct_hi = 0.4
# coords_list_hi = gjson["features"][list_ind_hi]["geometry"]["coordinates"]
# for ind_isl, island in enumerate(coords_list_hi):
#     for ind_pnt, point in enumerate(island[0]):
#         gjson["features"][list_ind_hi]["geometry"][
#             "coordinates"
#         ][ind_isl][0][ind_pnt][0] = point[0] + incr_pct_hi * (point[0] +
#                                                               min_abs_lon_hi)
#         gjson["features"][list_ind_hi]["geometry"][
#             "coordinates"
#         ][ind_isl][0][ind_pnt][1] = point[1] + incr_pct_hi * (point[1] -
#                                                               min_lat_hi)

# # Move Hawaii closer to the mainland such that the minimum minimum absolute
# # longitude and lattitude are (-125, 27)
# min_lat_hi_new = 27.5
# min_abs_lon_hi_new = 124.5
# for ind_isl, island in enumerate(coords_list):
#     for ind_pnt, point in enumerate(island[0]):
#         gjson["features"][list_ind_hi]["geometry"][
#             "coordinates"
#         ][ind_isl][0][ind_pnt][0] = point[0] + (min_abs_lon_hi -
#                                                 min_abs_lon_hi_new)
#         gjson["features"][list_ind_hi]["geometry"][
#             "coordinates"
#         ][ind_isl][0][ind_pnt][1] = point[1] - (min_lat_hi - min_lat_hi_new)

# # Add a state box around Delaware abbreviation DE
# st_list_num = 2
# de_coord_list = [gjson["features"][st_list_num]["geometry"]["coordinates"]]
# new_box_de = [[
#     [-75.4, 38.8],
#     [-72.4, 38.3],
#     [-72.4, 38.9],
#     [-70.2, 38.9],
#     [-70.2, 37.7],
#     [-72.4, 37.7],
#     [-72.4, 38.3]
# ]]
# de_coord_list.append(new_box_de)
# gjson["features"][st_list_num]["geometry"]["coordinates"] = de_coord_list
# gjson["features"][st_list_num]["geometry"]["type"] = "MultiPolygon"

# # Add a state box around Washington, DC (District of Columbia) abbreviation DC
# st_list_num = 35
# dc_coord_list = [gjson["features"][st_list_num]["geometry"]["coordinates"]]
# new_box_dc = [[
#     [-77.0, 38.9],
#     [-73.3, 35.5],
#     [-73.3, 36.1],
#     [-71.1, 36.1],
#     [-71.1, 34.9],
#     [-73.3, 34.9],
#     [-73.3, 35.5]
# ]]
# dc_coord_list.append(new_box_dc)
# gjson["features"][st_list_num]["geometry"]["coordinates"] = dc_coord_list
# gjson["features"][st_list_num]["geometry"]["type"] = "MultiPolygon"

# # Add a state box around Massachusetts abbreviation MD
# st_list_num = 0
# md_coord_list = gjson["features"][st_list_num]["geometry"]["coordinates"]
# new_box_md = [[
#     [-76.8, 39.3],
#     [-72.7, 37.0],
#     [-72.7, 37.6],
#     [-70.5, 37.6],
#     [-70.5, 36.4],
#     [-72.7, 36.4],
#     [-72.7, 37.0]
# ]]
# md_coord_list.append(new_box_md)
# gjson["features"][st_list_num]["geometry"]["coordinates"] = md_coord_list

# # Add a state box around Massachusetts abbreviation MA
# st_list_num = 29
# ma_coord_list = gjson["features"][st_list_num]["geometry"]["coordinates"]
# new_box_ma = [[
#     [-71.7, 42.2],
#     [-68.5, 42.2],
#     [-68.5, 42.8],
#     [-66.3, 42.8],
#     [-66.3, 41.6],
#     [-68.5, 41.6],
#     [-68.5, 42.2]
# ]]
# ma_coord_list.append(new_box_ma)
# gjson["features"][st_list_num]["geometry"]["coordinates"] = ma_coord_list

# # Add a state box around New Jersey abbreviation NJ
# st_list_num = 34
# nj_coord_list = [gjson["features"][st_list_num]["geometry"]["coordinates"]]
# new_box_nj = [[
#     [-74.4, 40.1],
#     [-72.0, 39.7],
#     [-72.0, 40.3],
#     [-69.8, 40.3],
#     [-69.8, 39.1],
#     [-72.0, 39.1],
#     [-72.0, 39.7]
# ]]
# nj_coord_list.append(new_box_nj)
# gjson["features"][st_list_num]["geometry"]["coordinates"] = nj_coord_list
# gjson["features"][st_list_num]["geometry"]["type"] = "MultiPolygon"

# # Add a state box around Rhode Island abbreviation RI
# st_list_num = 50
# ri_coord_list = gjson["features"][st_list_num]["geometry"]["coordinates"]
# new_box_ri = [[
#     [-71.5, 41.7],
#     [-69.5, 40.4],
#     [-69.5, 41.0],
#     [-67.3, 41.0],
#     [-67.3, 39.8],
#     [-69.5, 39.8],
#     [-69.5, 40.4]
# ]]
# ri_coord_list.append(new_box_ri)
# gjson["features"][st_list_num]["geometry"]["coordinates"] = ri_coord_list

# # Merge the state tax type data into gjson for each state
# state_amt_df
# for ind_st, state in enumerate(gjson["features"]):
#     st_abbrev = state["properties"]["STUSPS"]
#     state["properties"]["state_amt"] = state_amt_df[
#         state_amt_df["Abbrev"]==st_abbrev
#     ]["StateAMT_str_short"].iloc[0]

# amt_labels = [
#     'No state AMT', 'Has state AMT'
# ]
# amt_colors = ["white", "purple"]

# source_shapes = {}
# for category in amt_labels:
#     source_shapes[category] = {"type": "FeatureCollection", "features": []}

# for item in gjson["features"]:
#     source_shapes[item["properties"]["state_amt"]]['features'].append(item)

# TOOLS = "pan, box_zoom, wheel_zoom, hover, save, reset, help"

# fig1 = figure(
#     title=fig1_title,
#     height=500,
#     width=1050,
#     tools=TOOLS,
#     # tooltips=[
#     #     ("State", @state_names), ("Tax type", @tax_type)
#     # ]
#     # match_aspect = True,
#     min_border = 0,
#     x_axis_location = None, y_axis_location = None,
#     toolbar_location="right"
# )
# fig1.toolbar.logo = None
# fig1.grid.grid_line_color = None

# cmap = CategoricalColorMapper(
#     palette=amt_colors, factors=amt_labels
# )
# for category in amt_labels:
#     source_shape_1 = GeoJSONDataSource(
#         geojson = json.dumps(source_shapes[category])
#     )
#     fig1.patches(
#         'xs', 'ys', source=source_shape_1, fill_alpha=0.7,
#         fill_color = {'field': 'state_amt', 'transform': cmap},
#         line_color ='black', line_width=1.0, line_alpha=0.3,
#         hover_line_color="black", hover_line_width=3.0, legend_label=category)

#     hover = fig1.select_one(HoverTool)
#     hover.point_policy = "follow_mouse"
#     hover.tooltips = [
#         ("State", "@NAME"),
#         ("State AMT", "@state_amt")
#     ]

# # Add 2-letter state abbreviation labels. See Bokeh documentation for labels at
# # https://docs.bokeh.org/en/latest/docs/user_guide/basic/annotations.html.
# label_lon_lat =[
#     ["AL",  -87.40, 32.10],
#     ["AK", -135.00, 48.30],
#     ["AZ", -112.40, 34.00],
#     ["AR",  -93.20, 34.30],
#     ["CA", -121.00, 37.00],
#     ["CO", -106.50, 38.50],
#     ["CT",  -73.40, 41.15],
#     ["DE",  -72.10, 37.80],
#     ["DC",  -73.00, 35.00],
#     ["FL",  -82.40, 28.00],
#     ["GA",  -84.20, 32.20],
#     ["HI", -125.00, 29.00],
#     ["ID", -115.00, 43.00],
#     ["IL",  -89.60, 39.50],
#     ["IN",  -86.90, 39.60],
#     ["IA",  -94.40, 41.60],
#     ["KS",  -99.50, 38.00],
#     ["KY",  -86.00, 37.00],
#     ["LA",  -93.00, 30.20],
#     ["ME",  -70.00, 44.50],
#     ["MD",  -72.40, 36.50],
#     ["MA",  -68.20, 41.80],
#     ["MI",  -85.40, 42.70],
#     ["MN",  -95.30, 45.50],
#     ["MS",  -90.60, 32.10],
#     ["MO",  -93.50, 38.00],
#     ["MT", -110.50, 46.50],
#     ["NE", -100.50, 41.00],
#     ["NV", -118.00, 39.00],
#     ["NH",  -72.45, 42.70],
#     ["NM", -107.00, 34.00],
#     ["NJ",  -71.60, 39.20],
#     ["NY",  -76.00, 42.50],
#     ["NC",  -79.20, 35.00],
#     ["ND", -101.50, 46.80],
#     ["OH",  -83.80, 39.90],
#     ["OK",  -98.00, 35.00],
#     ["OR", -121.50, 43.50],
#     ["PA",  -78.50, 40.40],
#     ["RI",  -69.00, 39.90],
#     ["SC",  -81.50, 33.30],
#     ["SD", -101.00, 44.00],
#     ["TN",  -87.10, 35.40],
#     ["TX", -100.00, 31.00],
#     ["UT", -112.50, 39.00],
#     ["VT",  -73.20, 44.10],
#     ["VA",  -79.00, 37.10],
#     ["WA", -121.00, 47.00],
#     ["WV",  -81.90, 38.00],
#     ["WI",  -90.40, 44.00],
#     ["WY", -108.50, 42.50]
# ]
# label_abbrev = [state[0] for state in label_lon_lat]
# label_lon = [state[1] for state in label_lon_lat]
# label_lat = [state[2] for state in label_lon_lat]
# state_cds = ColumnDataSource(data=dict(
#     lon=label_lon,
#     lat=label_lat,
#     abbrev=label_abbrev
# ))

# state_labels = LabelSet(
#     x='lon', y='lat', text='abbrev', text_font_size="9pt",
#     text_font_style="bold", x_offset=0, y_offset=0, source=state_cds
# )

# fig1.add_layout(state_labels)

# # Legend properties
# fig1.legend.click_policy = 'mute'
# fig1.legend.location = "center_left"

# fig1.add_layout(
#     Title(
#         text="  Source: Richard W. Evans (@RickEcon), updated October 16, 2023.",
#         align="left",
#         text_font_size="3mm",
#         text_font_style="italic",
#     ),
#     "below"
# )
# show(fig1)

Loading BokehJS ...

PermissionError: [Errno 1] Operation not permitted